## spaCy Overview

spaCy is a Python library for NLP. "spaCy is designed specifically for production use and helps you build applications that process and “understand” large volumes of text. It can be used to build information extraction or natural language understanding systems, or to pre-process text for deep learning." Unlike many other NLP systems, spaCy is non-destructive (you don't lose any of the original text you put in). (https://spacy.io/usage/spacy-101)

"When you call nlp on a text, spaCy first tokenizes the text to produce a Doc object. The Doc is then processed in several different steps – this is also referred to as the processing pipeline. The pipeline used by the default models consists of a tagger, a parser and an entity recognizer. Each pipeline component returns the processed Doc, which is then passed on to the next component." First is the tokenizer, which segments text into tokens. Some other pipeline components are the `tagger` (part of speech tags), `parser` (dependency parsing, which is the default for sentence segmentation), `ner` (named entitiy recognizer), `textcat` (text categorizer), and any custom components the user decides to add. "The processing pipeline always depends on the statistical model and its capabilities. For example, a pipeline can only include an entity recognizer component if the model includes data to make predictions of entity labels." (https://spacy.io/usage/processing-pipelines#_title)

## Training spaCy's Statistical Models
https://spacy.io/usage/training

### Training basics
"spaCy’s models are statistical and every “decision” they make – for example, which part-of-speech tag to assign, or whether a word is a named entity – is a prediction. This prediction is based on the examples the model has seen during training. To train a model, you first need training data – examples of text, and the labels you want the model to predict... The model is then shown the unlabelled text and will make a prediction. Because we know the correct answer, we can give the model feedback on its prediction in the form of an error gradient of the loss function that calculates the difference between the training example and the expected output. The greater the difference, the more significant the gradient and the updates to our model."

### Simple training style
In the simple training style, you "pass **raw texts** and **dictionaries of annotations** to `nlp.update`. The dictionaries can have the keys `entities`, `heads`, `deps`, `tags` and `cats`."

"Once you’ve got your pipeline together and you want to tune the accuracy, you usually want to process your training examples in batches, and experiment with `minibatch` sizes and dropout rates, set via the `drop` keyword argument."

### Optimization tips and advice
There are lots of conflicting “recipes” for training deep neural networks at the moment. The cutting-edge models take a very long time to train, so most researchers can’t run enough experiments to figure out what’s really going on. For what it’s worth, here’s a recipe that seems to work well on a lot of NLP problems:

1. Initialize with batch size 1, and compound to a maximum determined by your data size and problem type.
2. Use Adam solver with fixed learning rate.
3. Use averaged parameters
4. Use L2 regularization.
5. Clip gradients by L2 norm to 1.
6. On small data sizes, start at a high dropout rate, with linear decay.

#### Compounding batch size
"In training the various spaCy models, we haven’t found much advantage from decaying the learning rate – but starting with a low batch size has definitely helped." (see the example at https://spacy.io/usage/training#tips-batch-size) 

"This will set the batch size to start at 1, and increase each batch until it reaches a maximum size. The tagger, parser and entity recognizer all take whole sentences as input, so they’re learning a lot of labels in a single example. You therefore need smaller batches for them. The batch size for the text categorizer should be somewhat larger, especially if your documents are long."

#### Learning rate, regularization, and gradient clipping
"By default spaCy uses the Adam solver, with default settings (`learn_rate=0.001, beta1=0.9, beta2=0.999`). Some researchers have said they found these settings terrible on their problems – but they’ve always performed very well in training spaCy’s models, in combination with the rest of our recipe. You can change these settings directly, by modifying the corresponding attributes on the optimizer object. You can also set environment variables, to adjust the defaults." 

"There are two other key hyper-parameters of the solver: **L2** regularization, and **gradient clipping** (`max_grad_norm`). Gradient clipping is a hack that’s not discussed often, but everybody seems to be using. It’s quite important in helping to ensure the network doesn’t diverge, which is a fancy way of saying “fall over during training”. The effect is sort of similar to setting the learning rate low. It can also compensate for a large batch size (this is a good example of how the choices of all these hyper-parameters intersect)."

#### Dropout rate
"For small datasets, it’s useful to set a high dropout rate at first, and decay it down towards a more reasonable value. This helps avoid the network immediately overfitting, while still encouraging it to learn some of the more interesting things in your data. spaCy comes with a `decaying` utility function to facilitate this." 

"All of the models currently use Bernoulli dropout, for no particularly principled reason – we just haven’t experimented with another scheme like Gaussian dropout yet."

#### Parameter averaging [TODO: ask]
"The trick is to store the moving average of the weights during training. We don’t optimize this average – we just track it. Then when we want to actually use the model, we use the averages, not the most recent value. In spaCy (and Thinc) this is done by using a context manager, `use_params`, to temporarily replace the weights"

"The context manager is handy because you naturally want to evaluate and save the model at various points during training (e.g. after each epoch). After evaluating and saving, the context manager will exit and the weights will be restored, so you resume training from the most recent value, rather than the average. By evaluating the model after each epoch, you can remove one hyper-parameter from consideration (the number of epochs). Having one less magic number to guess is extremely nice – so having the averaging under a context manager is very convenient."

## Relevant utility functions
https://spacy.io/api/top-level

- `spacy.prefer_gpu`: "Allocate data and perform operations on GPU, if available."
- `util.minibatch`: "Iterate over batches of items. size may be an iterator, so that batch-size can vary on each step." Yields a list (the batches).
- `util.compounding`: "Yield an infinite series of compounding values. Each time the generator is called, a value is produced by multiplying the previous value by the compound rate."
- `util.decaying`: "Yield an infinite series of linearly decaying values."
- `util.itershuffle`: "Shuffle an iterator. This works by holding `bufsize` items back and yielding them sometime later. Obviously, this is not unbiased – but should be good enough for batching. Larger `bufsize` means less bias."

## Relevant language functions
https://spacy.io/api/language

- `Language.pipe`: "Process texts as a stream, and yield Doc objects in order. This is usually more efficient than processing texts one-by-one." (Allows multiprocessing using the `n_process` keyword.)
- `Language.update`: "Update the models in the pipeline."
- `Language.evaluate`: "Evaluate a model's pipeline components."
- `Language.begin_training`: "Allocate models, pre-process training data and acquire an optimizer."
- `Language.use_params`: "Replace weights of models in the pipeline with those provided in the params dictionary. Can be used as a context manager, in which case, models go back to their original weights after the block."
- `Language.create_pipe`: "Create a pipeline component from a factory." (Takes a string name and an optional dictionary of configuration parameters.)
- `Language.add_pipe`: "Add a component to the processing pipeline. Valid components are callables that take a `Doc` object, modify it and return it."
- `Language.disable_pipes`: "Disable one or more pipeline components. If used as a context manager, the pipeline will be restored to the initial state at the end of the block. Otherwise, a `DisabledPipes` object is returned, that has a `.restore()` method you can use to undo your changes."
- `Language.to_disk`: "Save the current state to a directory. If a model is loaded, this will include the model."
- `Language.from_disk`: "Loads state from a directory. Modifies the object in place and returns it. If the saved Language object contains a model, the model will be loaded." Allows `exclude` keyword argument to indicate any pipeline components you don't want to load.

## TextCategorizer configuration parameters
Just the ones that are relevant to me. See https://spacy.io/api/textcategorizer.

- `exclusive_classes`: "Make categories mutually exclusive. Defaults to `False`."
- `architecture`: "Model architecture to use, see architechtures for details. Defaults to "**ensemble**"."
- `"ensemble"`: "Stacked ensemble of a bag-of-words model and a neural network model. The neural network uses a CNN with mean pooling and attention. The “ngram_size” and “attr” arguments can be used to configure the feature extraction for the bag-of-words model."
- `"simple_cnn"`: "A neural network model where token vectors are calculated using a CNN. The vectors are mean pooled and used as features in a feed-forward network. This architecture is usually less accurate than the ensemble, but runs faster."
- `"bow"`: "An ngram “bag-of-words” model. This architecture should run much faster than the others, but may not be as accurate, especially if texts are short. The features extracted can be controlled using the keyword arguments ngram_size and attr. For instance, ngram_size=3 and attr="lower" would give lower-cased unigram, trigram and bigram features. 2, 3 or 4 are usually good choices of ngram size."
- `token_vector_width`: Length of token vectors. Defaults to the enviornment variable, which has the default value of `128`. If the environment variable is not provided, a default of `96` is used. Used by `"simple_cnn"` architecture. (Missing from documentation but can be found in source code at https://github.com/explosion/spaCy/blob/master/spacy/pipeline/pipes.pyx)
- `embed_size`: an environment variable for the number of rows in embedding tables. (https://spacy.io/api/cli#train-hyperparams).

### More information from _ml.py
Information and configuration parameters you find in https://github.com/explosion/spaCy/blob/master/spacy/_ml.py as opposed to TextCategorizer documentation or https://github.com/explosion/spaCy/blob/master/spacy/pipeline/pipes.pyx.

- For `"simple_cnn"` architexture, "If `exclusive_classes=True`, a softmax non-linearity is applied, so that the outputs sum to 1. If `exclusive_classes=False`, a logistic non-linearity is applied instead, so that outputs are in the range [0, 1]." This is also true for the bag of words model.
- For `"bow"` architecture, it's a linear model using the extracted ngrams. As with `"simple_cnn"`, the output layer uses a softmax non-linearity if `exclusive_classes=True` and a logistic non-linearity otherwise.

#### Ensemble architecture
Source code in https://github.com/explosion/spaCy/blob/master/spacy/_ml.py

I found this too complicated to understand more than was in the documentation.

- Additional config parameters accepted:
  - `depth`: defaults to `2`.
  - `nr_vector`: defaults to `5000`.
  - `pretrained_dims`: defaults to `0`.
  - `width`: defaults to `64`. Named parameter.

## Notes on how to change things from tutorial
Assuming you start with the tutorial in https://github.com/explosion/spaCy/blob/master/examples/training/train_textcat.py

- to change optimizer, pass in the appropriate optimizer as keyword `sgd` to `nlp.begin_training()`
- to change batch size, change the `batch_sizes =` part
- to change dropout rate, change the `drop` keyword in `nlp.update` part (TODO: how to do dropout with linear decay?)
- `n_iter` in example appears to be epoch (`for i in range(n_iter)` you go through all the data).
- in `load_data`, the `split` keyword argument is the train-validation split as a percentage (e.g., `0.8` means 80% is train and 20% is validation).
- TODO: get `evaluate` working the way I want to (so I can show performance in each category)